# Analysis of S&P 500 returns for Alliance Bernstein
## Author: Ui-Wing Cheah

## Import statements

In [ ]:
import pandas as pd 
import scipy as sp 
import numpy as np
import matplotlib.pyplot as plt 
import ab_util as util 
from datetime import datetime,timedelta


## Processing the data

In [ ]:
# loading in the raw data
raw_data = pd.read_excel('spx_data.xlsx',index_col=0)
os_start = datetime(2009,12,31)
# we want to process the data into the following columns
# dates,prices,returns,year,tri,quint,dec,sample (0/1)
# keep the data in numerical format as much as possible

all_dts = raw_data.index.to_pydatetime()
data_df = raw_data

# computing returns
data_df['returns'] = data_df['prices'].divide(data_df['prices'].shift(1)).subtract(1)
# assigning year
data_df['year'] = pd.Series(index=all_dts,data=np.array([dt.year for dt in all_dts]))
data_df['sample'] = pd.Series(index=all_dts,data=0)
data_df['sample'][data_df.index>os_start]=1
data_df['daycount'] = pd.Series(index=all_dts,data=1.).cumsum()
# we drop the first date for convenience since there's no return
data_df = data_df.dropna(how='any',axis=0)

# we now take out the unique years
all_years = sorted(set(data_df['year'].values))

# we now break out the decades
dec_interval = all_years[9::10]
quint_interval = all_years[4::5]
tri_interval = all_years[2::3]

# label the decades
y0 = 1928
dec_series = pd.Series()
for iy,y in enumerate(dec_interval):
    ds_ = pd.Series(index = data_df[data_df['year']<=y][data_df['year']>y0].index,data=iy+1)
    dec_series = dec_series.combine_first(ds_)
    y0 =y
data_df['dec'] = dec_series

# label the 5-year points
y0 = 1928
quint_series = pd.Series()
for iy,y in enumerate(quint_interval):
    ds_ = pd.Series(index = data_df[data_df['year']<=y][data_df['year']>y0].index,data=iy+1)
    quint_series = quint_series.combine_first(ds_)
    y0 =y
data_df['quint'] = quint_series

# label the 3-year points
y0 = 1928
tri_series = pd.Series()
for iy,y in enumerate(tri_interval):
    ds_ = pd.Series(index = data_df[data_df['year']<=y][data_df['year']>y0].index,data=iy+1)
    tri_series = tri_series.combine_first(ds_)
    y0 =y
data_df['tri'] = tri_series


## Part 1: Comparing return distributions

In [ ]:
# we will focus on the just the 3Y samples 
tri_samples = sorted(set(data_df['tri'].values))
os_samples = [28,29,30]
sample_names = {28:'10-12',29:'13-15',30:'16-18'}

# getting the summary statistics of the 3Y data
sample_data_3Y = pd.DataFrame({i:util.get_sample_stats(data_df[data_df['tri']==i]['returns']) for i in tri_samples}).T


# computing the p-values of the sample averages and other statistic
sample_pval = pd.DataFrame({i:util.estimate_pvalue(sample_data_3Y.loc[i],sample_data_3Y) for i in os_samples}).rename(columns=sample_names)


In [ ]:
# plotting the central chart 
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(sample_data_3Y,os_samples=os_samples,plot_cols='center',scatter_names = sample_names,legend_dict={'loc':4},xlabel='Measure',ylabel='Daily Return')
ax_test.set_title('Fig 1: Central Returns',fontdict={'size':20})
fig_.savefig('charts\p1_ret_cent.png')

# plotting the dispersion chart 
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(sample_data_3Y,os_samples=os_samples,plot_cols='disp',scatter_names = sample_names,legend_dict={'loc':1},xlabel='Dispersion Measure',ylabel='Daily Return')
ax_test.set_title('Fig 2: Return Dispersion',fontdict={'size':20})
fig_.savefig('charts\p2_ret_disp.png')

# plotting the chart of tail returns
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(sample_data_3Y,os_samples=os_samples,scatter_names = sample_names,legend_dict={'loc':4},xlabel='Distribution Tail',ylabel='Daily Return')
ax_test.set_title('Fig 3: Tail Return Averages',fontdict={'size':20})
fig_.savefig('charts\p3_ret_tail.png')




## Part 2: Volatility Levels and Changes
Here we compute the scores of returns based on rolling estimates of volatility

In [ ]:
rets = data_df['returns']
min_obs = 63

vol_estimates = pd.DataFrame({'sigma_EW':rets.expanding(min_periods=min_obs,).std(),
'sigma_RW':rets.rolling(min_periods=min_obs,window=252).std(),
'sigma_HL2W':rets.ewm(halflife=10,min_periods=min_obs).std(),
'sigma_HL1M':rets.ewm(halflife=21,min_periods=min_obs).std(),
'sigma_HL3M':rets.ewm(halflife=63,min_periods=min_obs).std(),
'sigma_HL6M': rets.ewm(halflife=126,min_periods=min_obs).std()})

# compute square returns
vol_estimates['sqret'] = rets.pow(2)

# estimating volatilities
vol_changes = vol_estimates.subtract(vol_estimates.shift(1))
vol_changes = vol_changes.rename(columns={x:'volchg_{0}'.format(x.split('_')[-1]) for x in vol_estimates})

# computing vol adjustd returns
ret_scores = pd.DataFrame({'score_{0}'.format(x.split('_')[-1]):rets.divide(vol_estimates[x].shift(1)) for x in vol_estimates})



In [ ]:
# looking at the distribution of vol changes
# we will focus on 2W HL but we can always change this
chgvar = 'volchg_HL2W'

vol_changes = data_df.combine_first(vol_changes)
volch_data = pd.DataFrame({i:util.get_sample_stats(vol_changes[vol_changes['tri']==i][chgvar]) for i in tri_samples}).T
volch_pval = pd.DataFrame({i:util.estimate_pvalue(volch_data.loc[i],volch_data) for i in os_samples}).rename(columns=sample_names)


In [ ]:
# Plotting volatility forecasts and levels date
# vestimator = 'sigma_HL1M'
vestimator = 'sigma_HL2W'
vol_df = data_df.combine_first(vol_estimates)
vol_estim_samples = pd.DataFrame({i:util.get_sample_stats(vol_df[data_df['tri']==i][vestimator]) for i in tri_samples}).T
vol_sample_pval = pd.DataFrame({i:util.estimate_pvalue(vol_estim_samples.loc[i],vol_estim_samples) for i in os_samples}).rename(columns=sample_names)

# Plotting vol levels
# plotting the central chart 
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(vol_estim_samples,os_samples=os_samples,plot_cols='center',scatter_names=sample_names,legend_dict={'loc':2},xlabel='Measure of Volatility Estimates',ylabel='Daily Volatility')
ax_test.set_title('Fig 4: Central Volatility',fontdict={'size':20})
fig_.savefig('charts\p4_vol_cent.png')

# plotting the tails of volatility estimates
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(vol_estim_samples,os_samples=os_samples,scatter_names = sample_names,legend_dict={'loc':2},xlabel='Distribution Tail',ylabel='Daily Volatility')
ax_test.set_title('Fig 6: Tail Volatility',fontdict={'size':20}, )
fig_.savefig('charts\p6_vol_tail.png')





In [ ]:
# Plotting volatility forecasts and levels date
vchestimator = 'sigma_HL2W'
volch_df = data_df.combine_first(vol_changes)
volch_estim_samples = pd.DataFrame({i:util.get_sample_stats(volch_df[volch_df['tri']==i]['volchg_HL1M']) for i in tri_samples}).T
volch_sample_pval = pd.DataFrame({i:util.estimate_pvalue(volch_estim_samples.loc[i],volch_estim_samples) for i in os_samples}).rename(columns=sample_names)

# Plotting vol changes
# plotting the central chart 
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(volch_estim_samples,os_samples=os_samples,plot_cols='center',scatter_names=sample_names,legend_dict={'loc':3},xlabel='Measure of Volatility Changes',ylabel='Daily Volatility Change')
ax_test.set_title('Fig 5: Central Volatility Changes',fontdict={'size':20},)
fig_.savefig('charts\p5_volch_cent.png')

# plotting the tails of volatility changes
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(volch_estim_samples,os_samples=os_samples,scatter_names = sample_names,legend_dict={'loc':2},xlabel='Distribution Tail',ylabel='Daily Volatility Change')
ax_test.set_title('Fig 7: Tail Volatility Changes',fontdict={'size':20},)
fig_.savefig('charts\p7_volch_tail.png')




## PART 3: VOLATILITY SURPRISES

In [ ]:
# Estimating vol thresholds
# surprises are labeled as extremely negative (nnn), very negative (nn) , ...extremely positive (ppp)
# define the thresholds as per normal distribution
# use 1M halflife for this analysis
thresholds = [-2.33,-1.96,-1.65,1.65,1.96,2.33]
threshnames = {-2.33:'s_nnn',-1.96:'s_nn',-1.65:'s_n',1.65:'s_p',1.96:'s_pp',2.33:'s_ppp'}
vol_estim = ['HL1M',]
# defining some simple helper functions
def calc_th(x,th,dropfalse=False):
    if dropfalse:
        if th < 0:
            y = x[x<=th]
        else:
            y = x[x>=th]
        y = y*0+1
        return y

    else:
        y = x*0
        if th < 0:
            y[x<=th]=1
        else:
            y[x>=th]=1
        return y

# this computes the percentage of surprises relative to total sample size
def pctg_surprises(sdf,):
    pctg = {}
    for i in tri_samples:
        sdf_ = sdf[sdf['tri']==i].reindex(columns=['s_nnn','s_nn','s_n','s_p','s_pp','s_ppp'])
        pctg[i] = sdf_.sum().divide(len(sdf_.index))
    return pd.DataFrame(pctg).T

def med_surp_intervals(sdf,):
    surptime = {}
    cols = ['s_nnn','s_nn','s_n','s_p','s_pp','s_ppp']
    for j in cols:
        sdf_ = (sdf[j]*sdf['daycount']).dropna()
        surptime[j] = sdf_.subtract(sdf_.shift(1))
    surptime = data_df.combine_first(pd.DataFrame(surptime))
    # getting by columns = surpries
    surpint = {}
    for i in tri_samples:
        st_ = surptime[surptime['tri']==i].reindex(columns=cols)
        surpint[i] = st_.median()
    return pd.DataFrame(surpint).T


# creating a combined dataframe
vol_df = data_df.combine_first(ret_scores)
# add the day count
vol_df['day_count'] = pd.Series(index=vol_df.index,data=1).cumsum()
# we now compute the surprise indicators
vol_surp_perc = {}
vol_surp_int = {}

for estim in vol_estim:
    # getting the surprise count across thresholds 
    surp_count = pd.DataFrame({threshnames[th]:calc_th(ret_scores['score_{0}'.format(estim)],th,True) for th in thresholds})
    s_df = data_df.combine_first(surp_count)
    vol_surp_perc[estim] = pctg_surprises(s_df)
    
    # getting the surprise intervals
    surp_intervals = pd.DataFrame({threshnames[th]:calc_th(ret_scores['score_{0}'.format(estim)],th,True) for th in thresholds})
    s_df_ = data_df.combine_first(surp_intervals)
    vol_surp_int[estim] = med_surp_intervals(s_df_)

# creating vol surprises count dataframe
vol_surp_perc = pd.concat(vol_surp_perc)
vol_surp_pval = pd.concat({estim: pd.DataFrame({i:util.estimate_pvalue(vol_surp_perc.loc[estim].loc[i],vol_surp_perc.loc[estim]) for i in os_samples}).rename(columns=sample_names) for estim in vol_estim})

# creating surprise intervals dataframe
vol_surp_int = pd.concat(vol_surp_int)
vol_surp_int_pval = pd.concat({estim: pd.DataFrame({i:util.estimate_pvalue(vol_surp_int.loc[estim].loc[i],vol_surp_int.loc[estim]) for i in os_samples}).rename(columns=sample_names) for estim in vol_estim})

In [ ]:
# Plotting volatility forecasts and levels date
surpestimate = 'HL1M'

# plotting the tails of volatility changes
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(vol_surp_perc.loc['HL1M'],os_samples=os_samples, plot_cols=['s_nnn','s_nn','s_n','s_p','s_pp','s_ppp'],scatter_names=sample_names,legend_dict={'loc':2},xlabel='Surprise Score',ylabel='Proportion of suprises in sample')
ax_test.legend(loc=1,)
ax_test.set_title('Fig 8: Surprise Rate',fontdict={'size':20})
ax_test.set_xticklabels(['-2.33$\sigma$','-1.96$\sigma$','-1.65$\sigma$','1.65$\sigma$','1.96$\sigma$','2.33$\sigma$',])
fig_.savefig('charts\p8_volsurp_pctg.png')

# # plotting the central chart 
fig_ = plt.figure(figsize=(12,8))
ax_test = util.box_scatter(vol_surp_int.loc['HL1M'],os_samples=os_samples,plot_cols=['s_nnn','s_nn','s_n','s_p','s_pp','s_ppp'],scatter_names=sample_names, legend_dict={'loc':1}, xlabel='Surprise Score',ylabel='Days',)

ax_test.set_title('Fig 9: Surprise Intervals',fontdict={'size':20})
ax_test.set_xticklabels(['-2.33$\sigma$','-1.96$\sigma$','-1.65$\sigma$','1.65$\sigma$','1.96$\sigma$','2.33$\sigma$'])
fig_.savefig('charts\p9_volsurp_intv.png')

In [ ]:
# getting the percentile values here for formatting
pvals = pd.concat({'returns':sample_pval,'vols':vol_sample_pval,'volch':volch_sample_pval,'vol_surp_pctg':vol_surp_pval.loc['HL1M'],'vol_surp_int':vol_surp_int_pval.loc['HL1M']})
pvals.to_excel('pvals_copy.xlsx')

## Bootstrapping Analysis (Extra)

In [ ]:
# we will run a bootstrap here but probably put the results in the apppendix as an extra
# getting the bootstrap samples
bsample_3Y = util.bootstrap_by_year(data_df,ndraws=1000,ksample=3,)
# generating the bootstrap distribution
bdist_3Y = util.get_bootstrap_stats(bsample_3Y)
btstrp_pval = pd.DataFrame({i:util.estimate_pvalue(sample_data_3Y.loc[i],bdist_3Y) for i in os_samples}).rename(columns=sample_names)
